In [4]:
import cv2
import numpy as np
import random
import time
from tensorflow.keras.preprocessing import image


from tensorflow.keras.models import load_model
model = load_model("rock_paper_scissors_model.keras")

# -----------------------------
# Configuration
# -----------------------------
moves = ['rock', 'paper', 'scissors']
user_score = 0
comp_score = 0
rounds = 0
total_rounds = 10

# Load your trained model (replace with actual path if needed)
# model = tf.keras.models.load_model("saved_models/best_cat_dog_model.keras")
# Make sure it’s trained on rock-paper-scissors dataset with output layer of 3 classes
# model = your_model

font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(0)

# -----------------------------
# Helper: Predict Player Move
# -----------------------------
def predict_move(frame, model):
    img = cv2.resize(frame, (300, 300))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array, verbose=0)[0]
    return moves[np.argmax(pred)]

# -----------------------------
# Helper: Decide Winner
# -----------------------------
def get_winner(user, comp):
    if user == comp:
        return "Draw"
    elif (user == "rock" and comp == "scissors") or \
         (user == "paper" and comp == "rock") or \
         (user == "scissors" and comp == "paper"):
        return "You"
    else:
        return "Computer"

# -----------------------------
# Game Loop
# -----------------------------
while rounds < total_rounds:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    # Bounding box
    x1, y1, x2, y2 = 100, 100, 300, 300
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

    cv2.putText(frame, f"Round {rounds + 1}/{total_rounds}", (20, 40), font, 1, (255, 255, 255), 2)
    cv2.putText(frame, f"Score: You {user_score} - {comp_score} CPU", (20, 80), font, 0.8, (255, 255, 0), 2)

    cv2.imshow("Rock Paper Scissors", frame)
    cv2.waitKey(500)

    # Countdown
    for i in range(3, 0, -1):
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(frame, f"Show your move in {i}", (350, 150), font, 1, (0, 255, 255), 2)
        cv2.imshow("Rock Paper Scissors", frame)
        cv2.waitKey(1000)

    # Capture ROI and predict
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    roi = frame[y1:y2, x1:x2]
    user_move = predict_move(roi, model)

    # Computer move
    comp_move = random.choice(moves)
    winner = get_winner(user_move, comp_move)

    # Update score
    if winner == "You":
        user_score += 1
    elif winner == "Computer":
        comp_score += 1

    rounds += 1

    # -----------------------------
    # Display Results
    # -----------------------------
    display = frame.copy()
    cv2.rectangle(display, (x1, y1), (x2, y2), (0, 255, 0), 3)
    cv2.putText(display, f"Your Move: {user_move}", (350, 200), font, 1, (0, 255, 0), 2)
    cv2.putText(display, f"CPU Move: {comp_move}", (350, 250), font, 1, (255, 0, 0), 2)
    cv2.putText(display, f"Winner: {winner}", (350, 320), font, 1.2, (0, 0, 255), 3)
    cv2.putText(display, f"Score: You {user_score} - {comp_score} CPU", (20, 80), font, 0.8, (255, 255, 0), 2)

    cv2.imshow("Rock Paper Scissors", display)
    cv2.waitKey(2500)

# -----------------------------
# Final Winner Display
# -----------------------------
final_result = (
    "You Win the Game!" if user_score > comp_score
    else "Computer Wins the Game!" if comp_score > user_score
    else "It's a Draw!"
)

print(f"\nFinal Score: You {user_score} - {comp_score} CPU")
print(final_result)

for _ in range(60):
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    cv2.putText(frame, final_result, (100, 300), font, 1.2, (0, 0, 255), 3)
    cv2.putText(frame, f"Final Score: {user_score} - {comp_score}", (150, 350), font, 1, (255, 255, 255), 2)
    cv2.imshow("Rock Paper Scissors", frame)
    if cv2.waitKey(50) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()



Final Score: You 3 - 3 CPU
It's a Draw!
